In [7]:
import pandas as pd

df = pd.read_csv('../data/raw/unicorns_till_sep_2022.csv')
df.isnull().sum()

Company                                                                   0
Valuation ($B)                                                            0
Date Joined                                                               0
Country                                                                   0
City                                                                      0
Industry                                                                  0
Investors                                                                18
dtype: int64